# Spotify x Billboard Five Year Analysis Project: Data Analysis

(Description)

## Imports

### Packages

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

### Data

In [4]:
atacj = pd.read_csv('../data/AllTracksAndChartsJoined.csv')
atacj.head()

,Unnamed: 0,playlist_id,playlist_name,chart_position,album_release_date,release_year,release_date_datetime,album_id,album_artist,track_name,...,Metal,Jazz,Folk,Funk,Misc,Easy_listening,R&B,Reggae,Show_tunes,No Genre Assigned
0,0,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08,2021,2021-01-08,1qW1C4kDOXnrly22daHbxz,Morgan Wallen,Sand In My Boots,...,0,0,0,0,0,0,0,0,0,0
1,1,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08,2021,2021-01-08,1qW1C4kDOXnrly22daHbxz,Morgan Wallen,Wasted On You,...,0,0,0,0,0,0,0,0,0,0
2,2,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08,2021,2021-01-08,1qW1C4kDOXnrly22daHbxz,Morgan Wallen,Somebody’s Problem,...,0,0,0,0,0,0,0,0,0,0
3,3,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08,2021,2021-01-08,1qW1C4kDOXnrly22daHbxz,Morgan Wallen,More Surprised Than Me,...,0,0,0,0,0,0,0,0,0,0
4,4,3avCwQPH6DkhMTRsizon7N,Billboard 200 Top Albums 2021,1,2021-01-08,2021,2021-01-08,1qW1C4kDOXnrly22daHbxz,Morgan Wallen,865,...,0,0,0,0,0,0,0,0,0,0
